 **Module 2.5: Saving Custom Python Functions as Models** 
## 🎯 **Learning Objectives Expanded**

### 1️⃣ **Use `mlflow.pyfunc.PythonModel` to Define Custom Inference Logic**

* **What it means:**
  Creating a custom Python class that lets you define exactly how your model handles prediction requests, allowing you to include custom logic or rules beyond standard ML models.

* **Detailed Steps:**

  * Define a subclass of `mlflow.pyfunc.PythonModel`:

    ```python
    import mlflow.pyfunc

    class MyCustomModel(mlflow.pyfunc.PythonModel):
        def predict(self, context, model_input):
            # Custom prediction logic here
            return model_input.sum(axis=1)
    ```
  * The `predict` method is essential, and it defines how your model processes input data and produces predictions.

* **Why it matters:**
  Custom inference logic lets you handle specialized workflows, apply business rules, or combine multiple models in one consistent package.

---

### 2️⃣ **Package a Custom Model with Artifacts (e.g., Parameters, Files)**

* **What it means:**
  Including additional files or resources (known as artifacts) with your model, making them accessible during prediction.

* **Detailed Steps:**

  * Save your artifact files (e.g., scaler parameters, lookup tables, pipelines) to disk:

    ```python
    import joblib
    joblib.dump(pipeline, "pipeline.pkl")
    ```
  * Package these artifacts with your model in MLflow:

    ```python
    artifacts = {"pipeline": "pipeline.pkl"}
    mlflow.pyfunc.log_model(
        artifact_path="my_custom_model",
        python_model=MyCustomModel(),
        artifacts=artifacts
    )
    ```
  * Access artifacts within your model class using the `load_context` method:

    ```python
    def load_context(self, context):
        self.pipeline = joblib.load(context.artifacts["pipeline"])
    ```

* **Why it matters:**
  Artifacts ensure your custom model has all the necessary components, enabling consistent and accurate predictions wherever it runs.

---

### 3️⃣ **Log and Load the Model Using the Pyfunc Flavor**

* **What it means:**
  Saving your custom Python model and later loading it using MLflow’s `pyfunc` standard interface.

* **Detailed Steps:**

  * Log your model:

    ```python
    with mlflow.start_run():
        mlflow.pyfunc.log_model(
            artifact_path="custom_pyfunc",
            python_model=MyCustomModel(),
            artifacts=artifacts
        )
    ```
  * Load your model back from MLflow:

    ```python
    loaded_model = mlflow.pyfunc.load_model("runs:/<RUN_ID>/custom_pyfunc")
    ```

* **Why it matters:**
  The pyfunc flavor provides a standardized way to save and deploy custom logic across different platforms, ensuring reproducibility and ease of deployment.

---

### 4️⃣ **Run Predictions Using Arbitrary Python Logic**

* **What it means:**
  Using your loaded custom model to make predictions with any Python-based logic you've defined, even if it's not typical ML inference (e.g., combining results, applying thresholds, etc.).

* **Detailed Steps:**

  * After loading your model:

    ```python
    import pandas as pd
    input_data = pd.DataFrame({"feature1": [1, 2], "feature2": [3, 4]})
    predictions = loaded_model.predict(input_data)
    print(predictions)
    ```
  * Your custom logic within `predict()` will determine how input data is transformed and what results are returned.

* **Why it matters:**
  Running predictions with arbitrary logic allows complete flexibility to adapt to unique business rules or complex decision processes, greatly enhancing your model's applicability.



In [2]:
# 📓 Module 2.5: Saving Custom Python Functions as Models
# Goal: Learn how to wrap arbitrary Python logic in an MLflow model using the `pyfunc` flavor

# ✅ Step 1: Install MLflow
!pip install -q mlflow

# ✅ Step 2: Import required modules
import mlflow.pyfunc
import pandas as pd
import numpy as np

# ✅ Step 3: Define a custom PythonModel class
class AddMultiplierModel(mlflow.pyfunc.PythonModel):
    def load_context(self, context):
        with open(context.artifacts["multiplier"], "r") as f:
            self.multiplier = int(f.read())

    def predict(self, context, model_input):
        return model_input.apply(lambda x: x + self.multiplier)

# ✅ Step 4: Save multiplier to a text file as an artifact
multiplier_value = 5
with open("multiplier.txt", "w") as f:
    f.write(str(multiplier_value))

# ✅ Step 5: Define the model path and artifacts
model_path = "custom_pyfunc_model"
artifacts = {"multiplier": "multiplier.txt"}

# ✅ Step 6: Log the custom model
with mlflow.start_run():
    mlflow.pyfunc.log_model(
        artifact_path=model_path,
        python_model=AddMultiplierModel(),
        artifacts=artifacts
    )
    print("🔁 Custom pyfunc model logged.")

# ✅ Step 7: Load and use the model
loaded_model = mlflow.pyfunc.load_model(f"runs:/{mlflow.last_active_run().info.run_id}/{model_path}")

# Prepare test input
test_input = pd.Series([10, 20, 30])
result = loaded_model.predict(test_input)
print("\n✅ Prediction using custom pyfunc model:")
print(result)

/usr/local/lib/python3.11/dist-packages/mlflow/pyfunc/utils/data_validation.py:186: UserWarning: Add type hints to the `predict` method to enable data validation and automatic signature inference during model logging. Check https://mlflow.org/docs/latest/model/python_model.html#type-hint-usage-in-pythonmodel for more details.
  color_warning(
2025/07/30 20:09:06 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


2025/07/30 20:09:08 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🔁 Custom pyfunc model logged.



✅ Prediction using custom pyfunc model:
0    15
1    25
2    35
dtype: int64


## 📝 Assessment: Saving Custom Python Functions as Models

### 📘 Multiple Choice (Choose the best answer)

**1. What is `mlflow.pyfunc.PythonModel` used for?**    
A. To install Python packages in MLflow environments    
**B. To define custom Python logic for model inference** ✅    
C. To autolog standard Scikit-learn models    
D. To convert models to ONNX format    

---

**2. Which method must be implemented in a custom `PythonModel` class?**    
A. `__init__()`    
B. `fit()`    
**C. `predict()`** ✅    
D. `run()`    

---

**3. What does the `load_context()` method allow in a `PythonModel`?**    
A. To reload MLflow from disk    
**B. To load external artifacts (e.g., files or models)** ✅    
C. To save metrics    
D. To install dependencies    

---

**4. Which MLflow method is used to save a custom Python function-based model?**    
A. `mlflow.log_model()`    
**B. `mlflow.pyfunc.log_model()`** ✅    
C. `mlflow.save_pyfunc()`    
D. `mlflow.wrap_model()`    

---

### ✏️ Short Answer

**5. What is an artifact in the context of MLflow custom models?**    
*Answer: An external file (e.g., a parameter, config, or trained submodel) that is bundled with the model and loaded at runtime.*

---

**6. Why might someone choose to use a custom `pyfunc` model over standard flavors like `mlflow.sklearn`?**    
*Answer: For non-standard workflows, post-processing, business rules, or models built using unsupported libraries.*

---

### 🧪 Mini Project

**7. Task:**

* Create a new custom `PythonModel` that multiplies each input value by a configurable scale (read from a file)
* Save the scale in `scale.txt` and pass it as an artifact
* Log the model using `mlflow.pyfunc.log_model()`
* Load it back and test on a `pd.Series([1, 2, 3])`
* Print the prediction results

